In [248]:
import ast

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [249]:
df = pd.read_csv('DTP_DATA_2025.csv')
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
features_number = df.shape[1]
df.head(5)

C:\Users\Настя\AppData\Local\Temp\ipykernel_3660\4109715133.py:1: DtypeWarning: Columns (6,7,28,38,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('DTP_DATA_2025.csv')


(1475674, 91)


,DTPV,date,district,EMTP_NUMBER,CHOM,COORD_L,COORD_W,dor,dor_k,dor_z,...,num rear-wheel-drive vehicles,num four-wheel-drive vehicles,num guilty objects,share guilty vehicles,fatal traffic violation(s),num fatal traffic violations,bodily harm,guilty drivers' avg experience,drivers' avg experience,number fatal traffic violations
0,Столкновение,31.01.2015,ЖЕЛЕЗНОДОРОЖНЫЙ,NaN,Движение частично перекрыто,83.740833,53.333056,NaN,NaN,Не указано,...,0,0,1,0.5,{'Другие нарушения ПДД водителем'},0,"{'Раненый, находящийся (находившийся) на амбу...",15.0,13.0,1
1,Наезд на пешехода,31.01.2015,АЛЕЙСКИЙ,NaN,Режим движения не изменялся,82.462778,52.245833,А-322 Барнаул - Рубцовск - граница с Республик...,3.0,Федеральная (дорога федерального значения),...,0,0,1,1.0,{'Нахождение на проезжей части без цели её пер...,0,"{'Раненый, находящийся (находившийся) на стаци...",NaN,5.0,1
2,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,NaN,Движение частично перекрыто,83.501944,53.274167,NaN,NaN,Не указано,...,0,0,2,2.0,{'Переход через проезжую часть вне пешеходного...,0,"{'Раненый, находящийся (находившийся) на амбу...",NaN,17.0,1
3,Наезд на пешехода,31.01.2015,ЦЕНТРАЛЬНЫЙ,NaN,Режим движения не изменялся,83.251667,53.285278,NaN,NaN,Не указано,...,0,0,1,1.0,{'Переход через проезжую часть вне пешеходного...,0,"{'Раненый, находящийся (находившийся) на амбу...",NaN,27.0,1
4,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,NaN,Движение частично перекрыто,83.708889,53.318333,NaN,NaN,Не указано,...,0,0,1,1.0,{'Переход через проезжую часть в неустановленн...,0,"{'Раненый, находящийся (находившийся) на стаци...",NaN,1.0,1


In [250]:
df.columns

Index(['DTPV', 'date', 'district', 'EMTP_NUMBER', 'CHOM', 'COORD_L', 'COORD_W',
       'dor', 'dor_k', 'dor_z', 'factor', 'house', 'k_ul', 'km', 'm', 'NP',
       'ndu', 'OBJ_DTP', 'osv', 's_dtp', 's_pch', 'sdor', 'spog', 'street',
       'color', 'f_sob', 'g_v', 'm_pov', 'm_ts', 'marka_ts', 'n_ts', 'o_pf',
       'r_rul', 't_n', 't_ts', 'ts_s', 'ALCO', 'INJURED_CARD_ID', 'k_UCH',
       'NPDD', 'n_UCH', 'POL', 'SAFETY_BELT', 'SOP_NPDD', 's_SEAT_GROUP',
       's_SM', 's_T', 'v_ST', 'KTS', 'KUCH', 'kartId', 'POG', 'RAN', 'rowNum',
       'time', 'any TV technical failures', 'any non-private vehicles',
       'any russian vehicles', 'any white vehicles', 'any black vehicles',
       'any colored vehicles', 'any drunk drivers', 'any female drivers',
       'any escaped participants', 'any concomitant traffic violations',
       'any injuries without safety belt', 'num drunk participants',
       'num child restraints', 'num cyclists', 'num pedestrians',
       'min vehicles age', 'max ve

In [251]:
df = df.drop(['color', 'f_sob', 'g_v', 'm_pov', 'm_ts', 'marka_ts', 'n_ts', 'o_pf',
       'r_rul', 't_n', 't_ts', 'ts_s', 'ALCO', 'INJURED_CARD_ID', 'k_UCH',
       'NPDD', 'n_UCH', 'POL', 'SAFETY_BELT', 'SOP_NPDD', 's_SEAT_GROUP',
       's_SM', 's_T', 'v_ST'], axis=1)

# dropped features because they were accidentally included in the DataFrame; 
# they contain information about the last accident participant and the last vehicle in the XML

In [252]:
# just because i didn't think ahead

new_column_names = {
    "guilty drivers' avg experience": "guilty drivers avg experience",
    "drivers' avg experience": "drivers avg experience"
}

df = df.rename(columns=new_column_names)

In [253]:
df.columns

Index(['DTPV', 'date', 'district', 'EMTP_NUMBER', 'CHOM', 'COORD_L', 'COORD_W',
       'dor', 'dor_k', 'dor_z', 'factor', 'house', 'k_ul', 'km', 'm', 'NP',
       'ndu', 'OBJ_DTP', 'osv', 's_dtp', 's_pch', 'sdor', 'spog', 'street',
       'KTS', 'KUCH', 'kartId', 'POG', 'RAN', 'rowNum', 'time',
       'any TV technical failures', 'any non-private vehicles',
       'any russian vehicles', 'any white vehicles', 'any black vehicles',
       'any colored vehicles', 'any drunk drivers', 'any female drivers',
       'any escaped participants', 'any concomitant traffic violations',
       'any injuries without safety belt', 'num drunk participants',
       'num child restraints', 'num cyclists', 'num pedestrians',
       'min vehicles age', 'max vehicles age', 'avg vehicles age',
       'num A-class vehicles', 'num B-class vehicles', 'num C-class vehicles',
       'num D-class vehicles', 'num E-class vehicles', 'num S-class vehicles',
       'num non-car vehicles', 'num front-wheel-drive vehi

In [254]:
print(f"Number of observations before processing: {df.shape[0]}")

Number of observations before processing: 1475674


### Meanings of columns

In [255]:
column_meanings = {
    'DTPV': 'RTA type',
    # 'date',
    # 'district',
    'EMTP_NUMBER': 'RTA number', # its meaning is similar to kartId
    'CHOM': 'info about changes in traffic',
    'dor': 'road name',
    'dor_k': 'road category',
    'dor_z': 'road significance',
    'factor': 'factors influencing the traffic',
    # 'house',
    'k_ul': 'street significance',
    'km': 'RTA road km',
    'm': 'RTA road m',
    'NP': 'human settlement',
    'ndu': 'defects of the road network transport and operational maintenance',
    'OBJ_DTP': 'road network objects near RTA',
    'osv': 'lighting',
    's_dtp': '', # dk what is it 
    's_pch': 'road surface condition',
    'sdor': 'road network objects at the scene of RTA',
    'spog': 'weather',
    # 'street',
    # -----------------------------------------------------------------------------
    # (info about every vehicle, was not save as column)
    'color': 'vehicle color',
    'f_sob': 'form of ownership', # it is not displayed in pdf version
    'g_v': 'vehicle year of release',
    'm_pov': 'vehicle damage place',
    'm_ts': 'vehicle model',
    'marka_ts': 'vehicle brand',
    'n_ts': 'vehicle number in RTA',
    'o_pf': 'form of ownership', # similar to f_sob in meaning, but has other values, f_sob was preferred it
    'r_rul': 'steering position',
    't_n': 'vehicle technical failure',
    't_ts': 'vehicle class',
    'ts_s': 'info about vehicle leaving a RTA place',
    # (info about every participant, was not save as column)
    'ALCO': 'ppm',
    'INJURED_CARD_ID': '', # dk what is it
    'k_UCH': 'participant category',
    'NPDD': 'direct traffic violations',
    'n_UCH': 'participant number',
    'POL': 'gender',
    'SAFETY_BELT': 'safety belt',
    'SOP_NPDD': 'concomitant traffic violations',
    's_SEAT_GROUP': 'type of child restraint',
    's_SM': 'info about participant leaving a RTA place',
    's_T': 'severity of the consequences',
    'v_ST': 'driving experience',
    # ---------------------------------------------------------------------
    # (general RTA info again)
    'KTS': 'number of vehicle in RTA',
    'KUCH': 'number of participants in RTA',
    'kartId': 'RTA ID', # it is not displayed in pdf version
    'POG': 'number of deaths',
    'RAN': 'number of wounded',
    # 'rowNum',
    # 'time'
}

## Processing the data gaps

In [256]:
features_with_gaps = df.columns[df.isna().any()].tolist()
# print(features_with_gaps)

for i, col in enumerate(features_with_gaps, start=1):
    print(f"{i}. {col}")

1. EMTP_NUMBER
2. CHOM
3. COORD_L
4. COORD_W
5. dor
6. dor_k
7. house
8. k_ul
9. km
10. m
11. NP
12. street
13. min vehicles age
14. max vehicles age
15. avg vehicles age
16. guilty drivers avg experience


We don't need every feature, so let's create a list of columns to drop and update it as needed.

In [257]:
drop_columns = ['district', 'house', 'street', 'rowNum', 'km', 'm']
# drop_columns = ['district', 'house', 'street', 'rowNum', 'm']
# will be dropped because they are irrelevant to our task

__EMTP_NUMBER vs kartId__: `kartId` was chosen as an accident ID

`kartId` does not appear in the PDF but is present in the XML and serves the same function as `EMTP_NUMBER`, so we need to choose between them.

In [258]:
print(f"EMTP_NUMBER gaps: {df['EMTP_NUMBER'].isna().sum()}")
print(f"kartId gaps: {df['kartId'].isna().sum()}")

# drop EMTP_NUMBER, it's fulled of nan

drop_columns.append('EMTP_NUMBER')

EMTP_NUMBER gaps: 823047
kartId gaps: 0


__CHOM__: 2 objects were dropped

In [259]:
print(f"CHOM gaps: {df['CHOM'].isna().sum()}")

df = df[df['CHOM'].notna()]
print(f"Objects left: {df.shape[0]}")

CHOM gaps: 2
Objects left: 1475672


__COORD_L and COORD_W:__

In [260]:
df[['COORD_L', 'COORD_W']].dtypes

# turn into float64
df['COORD_L'] = pd.to_numeric(df['COORD_L'], errors='coerce')
df['COORD_W'] = pd.to_numeric(df['COORD_W'], errors='coerce')

df[['COORD_L', 'COORD_W']].dtypes

COORD_L    float64
COORD_W    float64
dtype: object

Begin with analyzing columns gaps: 15 objects were dropped

In [261]:
L_ind = set(df[df['COORD_L'].isna()]['kartId'])
W_ind = set(df[df['COORD_W'].isna()]['kartId'])
common_gaps = L_ind | W_ind

print(f"COORD_L gaps: {df['COORD_L'].isna().sum()}")
print(f"COORD_W gaps: {df['COORD_W'].isna().sum()}")
print(f"COORD_L and COORD_W common gaps: {len(common_gaps)}")

df = df[(df['COORD_L'].notna()) & (df['COORD_W'].notna())]
print(f"COORD_L gaps: {df['COORD_L'].isna().sum()}")
print(f"COORD_W gaps: {df['COORD_W'].isna().sum()}")
print(f"Objects left: {df.shape[0]}")

COORD_L gaps: 166
COORD_W gaps: 156
COORD_L and COORD_W common gaps: 167
COORD_L gaps: 0
COORD_W gaps: 0
Objects left: 1475505


### **dor, dor_k:** key features in our analysis

__dor and dor_k:__ 930.003 common gaps

Why `dor` is a key feature?
- contains road names
- will allow to identify accidents on toll roads (here you can find [the list of toll roads in Russia](https://avtodor-tr.ru/road/))
- so we can create binary key feature `toll road`

In [262]:
print(f"Gaps in dor: {df['dor'].isna().sum()}")
print(f"Gaps in dor_k: {df['dor_k'].isna().sum()}")

dor_ind = set(df[df['dor'].isna()]['kartId'])
dor_k_ind = set(df[df['dor_k'].isna()]['kartId'])
common_gaps = dor_ind | dor_k_ind

print(f"Dor and dor_k common gaps: {len(common_gaps)}")

Gaps in dor: 964910
Gaps in dor_k: 971592
Dor and dor_k common gaps: 977475


In [263]:
# now we are not going to fill the gaps (do it later), so drop NaN

df = df[df['dor'].notna()]
print(f"Observations left after deleting NaN in 'dor' feature: {df.shape[0]}")

Observations left after deleting NaN in 'dor' feature: 510595


Don't drop NaN for flexibility in further work.

__k_ul:__ replace nan by new value

In [264]:
print(f"k_ul gaps: {df['k_ul'].isna().sum()}")
print(f"street gaps: {df['street'].isna().sum()}")

# Can't try to fill the k_ul gaps by street values, cause the number of nan in street is much bigger.
# Replace nan by 'Назначение неизвестно'

df['k_ul'] = df['k_ul'].fillna('Назначение неизвестно')
print(f"k_ul gaps left: {df['k_ul'].isna().sum()}")

k_ul gaps: 127857
street gaps: 472593
k_ul gaps left: 0


__guilty drivers' average experience:__ 145498 objects were deleted

In [265]:
print(f"guilty drivers avg experience gaps: {df['guilty drivers avg experience'].isna().sum()}")

df = df[df['guilty drivers avg experience'].notna()]
print(f"Objects left: {df.shape[0]}")

guilty drivers avg experience gaps: 37401
Objects left: 473194


__min vehicles age, max vehicles age, avg vehicles age:__ will be filled by ML algorithm later

In [266]:
print(f"Gaps in min vehicles age: {df['min vehicles age'].isna().sum()}")
print(f"Gaps in max vehicles age: {df['max vehicles age'].isna().sum()}")
print(f"Gaps in avg vehicles age: {df['avg vehicles age'].isna().sum()}")

min_ind = set(df[df['min vehicles age'].isna()]['kartId'])
max_ind = set(df[df['max vehicles age'].isna()]['kartId'])
avg_ind = set(df[df['avg vehicles age'].isna()]['kartId'])
common_gaps = min_ind | max_ind | avg_ind

print(f"Common gaps: {len(common_gaps)}")

Gaps in min vehicles age: 6208
Gaps in max vehicles age: 6208
Gaps in avg vehicles age: 6208
Common gaps: 6208


Wi will train a model that will predict missing values ​​based on other features below.

#### **Lyrical digression and NP:** why it is useless and will be deleted

In [267]:
print(f"NP gaps: {df['NP'].isna().sum()}")

NP gaps: 358663


`NP` contains only (compared to `dor`) 388.721 gaps and had a potential to be a key feature.

Why `NP` could be a key feature?
- the aim of the thesis is not only to analyze toll roads but also to compare them with free roads in Russia
- toll roads in Russia are mostly highways, which makes them very different from accidents occurring within cities (due to different conditions, speed limits, etc.)
- for a correct analysis, accidents that occurred in populated areas must be excluded from the DataFrame
- identifying these areas using NP could help achieve this

Why it is NOT?
- because the feature doesn't work tha way it should
- according to its intended meaning, `NP` should either: 

(1) contain names of populated areas with non-populated areas represented as NaN or 

(2) include names of populated areas and a clear label for non-populated areas (e.g., "not NP" or "highway")

- in reality, `NP` contains both names of populated areas and labels for non-populated areas, but these labels account for less than 0.2% of the non-NaN observations in `NP`
- as a result, the column primarily reflects the names of populated areas nearest to the accident location — even when the accident occurred on a highway
- this makes the feature completely useless for the analysis objectives, as it does not reliably distinguish between accidents in populated and non-populated areas

In [268]:
np_marks_list = [
    'г ', 'д ', 'п ', 'с ', 'х ', 'ст-ца ', 'аал ', 'аул ', 'массив ', 'мкр ', 
    'пгт ', 'починок ', 'сл ', 'ст ', 'поселение ', 'жилрайон ', 'городок ', 'заимка ', 'м ', 'у'
    ]           # compiled by the method of analyzing unique 'NP' feature values

non_missing_np = df[df['NP'].notna()]
not_np_unique = non_missing_np['NP'].unique().tolist()
not_np_unique = sorted(not_np_unique)

for np in not_np_unique[:]:
    for np_mark in np_marks_list:
        if np_mark in np:
            not_np_unique.remove(np)
            break

non_NP_obs = df[df['NP'].isin(not_np_unique)]
print(f"Number of observations without gaps in 'NP', where 'NP' values are not considered as populated areas: {non_NP_obs.shape[0]}")
print('-----')
print(' ')

# and if we try to find any highway in this DataFrame we'll fail:

dor_uniq = df['dor'].unique().tolist()
M_1_dors = list()

for dor in dor_uniq:
    try:
        if 'М-1 ' in dor and 'подъездная' not in dor and 'Подъездная' not in dor:
            M_1_dors.append(dor)
    except TypeError:
        continue

M1_df = non_NP_obs[non_NP_obs['dor'].isin(M_1_dors)]
print(f"M-1 highway found in non_NP_obs DataFrame: {M1_df.shape[0]}")
print('Spoiler: the number will be the same for every toll road')
print('-----')
print(' ')

# and in full DataFrame:

M1_df = df[df['dor'].isin(M_1_dors)]
print(f"M-1 highway found in full DataFrame: {M1_df.shape[0]}")
print('-----')
print(' ')

# analyzing the 'NP' values of M-1 toll road observations and finding out the 'NP' uselesness

print('The M-1 toll road unique NP values (M-1 road is a highway):')
M1_df[M1_df['NP'].notna()]['NP'].unique()

Number of observations without gaps in 'NP', where 'NP' values are not considered as populated areas: 275
-----
 
M-1 highway found in non_NP_obs DataFrame: 0
Spoiler: the number will be the same for every toll road
-----
 
M-1 highway found in full DataFrame: 1642
-----
 
The M-1 toll road unique NP values (M-1 road is a highway):


array(['г Москва', 'г Кубинка', 'д Ляхово', 'с/п Новоивановское',
       'с/п Успенское', 'рп Новоивановское', 'д Барвиха', 'д Бутынь',
       'г Одинцово', 'г Клин', 'дп Лесной Городок', 'д Немчиново',
       'д Вырубово', 'с Жаворонки', 'д Красная Горка', 'д Каменка',
       'г Ярцево', 'д Анохово', 'г Гагарин'], dtype=object)

In [269]:
drop_columns.append('NP')

## Processing categorical features

Which features are categorical?

In [270]:
cat_cols = df.select_dtypes(include=['object']).columns
non_cat_cols = ['COORD_L', 'COORD_W', 'date', 'time']
non_cat_cols.append('dor')      # road names will be necessary on further steps
cat_cols = list(set(cat_cols) - set(drop_columns) - set(non_cat_cols))      # do not proceed unnecessary features

for i, col in enumerate(cat_cols, start=1):
    meaning = column_meanings.get(col, "...")     # there was a dct with column meanings above
    print(f"{i}. {col} — {meaning}")

1. OBJ_DTP — road network objects near RTA
2. bodily harm — ...
3. ndu — defects of the road network transport and operational maintenance
4. osv — lighting
5. fatal traffic violation(s) — ...
6. factor — factors influencing the traffic
7. s_pch — road surface condition
8. DTPV — RTA type
9. k_ul — street significance
10. dor_z — road significance
11. sdor — road network objects at the scene of RTA
12. spog — weather
13. CHOM — info about changes in traffic


We will handle each categorical feature manually to ensure consistent labeling and avoid issues caused by duplicates, typos, or overly detailed values that should be grouped together.

---

**1. dor_z — road significance**

In [271]:
sorted(df['dor_z'].unique())

['Другие места',
 'Иная дорога',
 'Местного значения (дорога местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)',
 'Местного значения (дороги местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)',
 'Не указано',
 'Региональная или межмуниципальная (дорога регионального или межмуниципального значения)',
 'Федеральная (дорога федерального значения)',
 'Частная (дорога, относящиеся к частной и иным формам собственности)',
 'Частная (дороги, относящиеся к частной и иным формам собственности)']

In [272]:
replace_map = {
    'Региональная или межмуниципальная (дорога регионального или межмуниципального значения)': 'Региональная или межмуниципальная',
    'Федеральная (дорога федерального значения)': 'Федеральная',
    'Местного значения (дороги местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)': 'Местного значения',
    'Местного значения (дорога местного значения, включая относящиеся к собственности поселений, муниципальных районов, городских округов)': 'Местного значения',
    'Частная (дороги, относящиеся к частной и иным формам собственности)': 'Частная',
    'Частная (дорога, относящиеся к частной и иным формам собственности)': 'Частная',
    'Другие места': 'Другое',
    'Иная дорога': 'Другое',
    'Не указано': 'Не указано'
}

df['dor_z'] = df['dor_z'].replace(replace_map)
sorted(df['dor_z'].unique())

['Другое',
 'Местного значения',
 'Не указано',
 'Региональная или межмуниципальная',
 'Федеральная',
 'Частная']

In [273]:
le = LabelEncoder()
df['dor_z_cat'] = le.fit_transform(df['dor_z'])

---

**2. ndu — defects of the road network transport and operational maintenance**

In [274]:
sorted(df['ndu'].unique())

['Дефекты покрытия',
 'Иные недостатки',
 'Нарушения в размещении наружной рекламы',
 'Не установлены',
 'Недостатки зимнего содержания',
 'Недостаточное освещение',
 'Неисправное освещение',
 'Неисправность светофора',
 'Неправильное применение, плохая видимость дорожных знаков',
 'Неровное покрытие',
 'Несоответствие дорожных ограждений предъявляемым требованиям',
 'Несоответствие железнодорожного переезда предъявляемым требованиям',
 'Несоответствие люков смотровых колодцев и ливневой канализации предъявляемым требованиям',
 'Неудовлетворительное состояние обочин',
 'Неудовлетворительное состояние разделительной полосы',
 'Низкие сцепные качества покрытия',
 'Ограничение видимости',
 'Отклонение верха головки рельса трамвайных (железнодорожных) путей, расположенных в пределах проезжей части, относительно покрытия, более чем на 2,0 см',
 'Отсутствие временных ТСОД в местах проведения работ',
 'Отсутствие дорожных знаков в необходимых местах',
 'Отсутствие дорожных ограждений в необхо

In [275]:
replace_map = {
    # Lighting and traffic lights
    'Недостаточное освещение': 'Освещение',
    'Неисправное освещение': 'Освещение',
    'Отсутствие освещения': 'Освещение',
    'Плохая видимость светофора': 'Освещение',
    'Неисправность светофора': 'Освещение',

    # Pavement and roadway
    'Дефекты покрытия': 'Покрытие',
    'Неровное покрытие': 'Покрытие',
    'Низкие сцепные качества покрытия': 'Покрытие',
    'Сужение проезжей части, наличие препятствий, затрудняющих движение транспортных средств': 'Покрытие',
    'Отклонение верха головки рельса трамвайных (железнодорожных) путей, расположенных в пределах проезжей части, относительно покрытия, более чем на 2,0 см': 'Покрытие',

    # Markup
    'Отсутствие, плохая различимость вертикальной разметки': 'Разметка',
    'Отсутствие, плохая различимость горизонтальной разметки проезжей части': 'Разметка',

    # Signs
    'Отсутствие дорожных знаков в необходимых местах': 'Дорожные знаки',
    'Неправильное применение, плохая видимость дорожных знаков': 'Дорожные знаки',

    # Fences and devices
    'Отсутствие дорожных ограждений в необходимых местах': 'Ограждения',
    'Отсутствие пешеходных ограждений в необходимых местах': 'Ограждения',
    'Плохая видимость световозвращателей, размещенных на дорожных ограждениях': 'Ограждения',
    'Отсутствие направляющих устройств и световозвращающих элементов на них': 'Ограждения',

    # Sidewalks, bus stops
    'Отсутствие тротуаров (пешеходных дорожек)': 'Пешеходная инфраструктура',
    'Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта': 'Пешеходная инфраструктура',

    # Road shoulders and median strips
    'Неудовлетворительное состояние обочин': 'Обочины и разделительные полосы',
    'Неудовлетворительное состояние разделительной полосы': 'Обочины и разделительные полосы',

    # Railway, manholes, crossings
    'Несоответствие железнодорожного переезда предъявляемым требованиям': 'Переезды и ж/д инфраструктура',
    'Несоответствие люков смотровых колодцев и ливневой канализации предъявляемым требованиям': 'Люки и колодцы',
    
    # Other
    'Отсутствие временных ТСОД в местах проведения работ': 'Отсутствие ТСОД',
    'Нарушения в размещении наружной рекламы': 'Наружная реклама',
    'Недостатки зимнего содержания': 'Зимнее содержание',
    'Несоответствие дорожных ограждений предъявляемым требованиям': 'Ограждения',
    'Ограничение видимости': 'Видимость',
    'Иные недостатки': 'Прочее',
    'Не установлены': 'Не указано'
}

df['ndu'] = df['ndu'].replace(replace_map)
sorted(df['ndu'].unique())

['Видимость',
 'Дорожные знаки',
 'Зимнее содержание',
 'Люки и колодцы',
 'Наружная реклама',
 'Не указано',
 'Обочины и разделительные полосы',
 'Ограждения',
 'Освещение',
 'Отсутствие ТСОД',
 'Переезды и ж/д инфраструктура',
 'Пешеходная инфраструктура',
 'Покрытие',
 'Прочее',
 'Разметка']

In [276]:
le = LabelEncoder()
df['ndu_cat'] = le.fit_transform(df['ndu'])

---
**3. CHOM — info about changes in traffic**

In [277]:
sorted(df['CHOM'].unique())

['Движение полностью перекрыто',
 'Движение частично перекрыто',
 'Режим движения не изменялся']

In [278]:
le = LabelEncoder()
df['CHOM_cat'] = le.fit_transform(df['CHOM'])

---
**4. s_pch — road surface condition**

In [279]:
sorted(df['s_pch'].unique())

['Гололедица',
 'Загрязненное',
 'Залитое (покрытое) водой',
 'Заснеженное',
 'Мокрое',
 'Не установлено',
 'Обработанное противогололедными материалами',
 'Пыльное',
 'Свежеуложенная поверхностная обработка',
 'Со снежным накатом',
 'Сухое']

In [280]:
replace_map = {
    'Сухое': 'Сухое',
    
    'Мокрое': 'Мокрое',
    'Залитое (покрытое) водой': 'Мокрое',
    
    'Заснеженное': 'Заснеженное',
    'Со снежным накатом': 'Заснеженное',
    'Гололедица': 'Гололед',
    
    'Обработанное противогололедными материалами': 'Обработанное',
    
    'Пыльное': 'Пыльное',
    'Загрязненное': 'Пыльное',
    
    'Свежеуложенная поверхностная обработка': 'Обработка',
    
    'Не установлено': 'Не указано'
}

df['s_pch'] = df['s_pch'].replace(replace_map)
sorted(df['s_pch'].unique())

['Гололед',
 'Заснеженное',
 'Мокрое',
 'Не указано',
 'Обработанное',
 'Обработка',
 'Пыльное',
 'Сухое']

In [281]:
le = LabelEncoder()
df['s_pch_cat'] = le.fit_transform(df['s_pch'])

---

__5. DTPV — RTA type__

In [282]:
sorted(df['DTPV'].unique())

['Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.',
 'Иной вид ДТП',
 'Наезд на велосипедиста',
 'Наезд на внезапно возникшее препятствие',
 'Наезд на гужевой транспорт',
 'Наезд на животное',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ',
 'Наезд на пешехода',
 'Наезд на препятствие',
 'Наезд на стоящее ТС',
 'Опрокидывание',
 'Отбрасывание предмета',
 'Падение груза',
 'Падение пассажира',
 'Столкновение',
 'Съезд с дороги']

In [283]:
replace_map = {
    # Collisions
    'Столкновение': 'Столкновение',
    'Опрокидывание': 'Столкновение',
    'Съезд с дороги': 'Столкновение',

    # Collisions with pedestrians and people
    'Наезд на пешехода': 'Наезд на пешехода',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность': 'Наезд на человека',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы': 'Наезд на человека',
    'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ': 'Наезд на человека',
    'Наезд на велосипедиста': 'Наезд на велосипедиста',

    # Collisions with objects and obstacles
    'Наезд на препятствие': 'Наезд на препятствие',
    'Наезд на стоящее ТС': 'Наезд на препятствие',
    'Наезд на внезапно возникшее препятствие': 'Наезд на препятствие',

    # Other raids
    'Наезд на животное': 'Наезд на животное',
    'Наезд на гужевой транспорт': 'Наезд на животное',

    # Other
    'Падение пассажира': 'Прочее',
    'Падение груза': 'Прочее',
    'Отбрасывание предмета': 'Прочее',
    'Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.': 'Прочее',
    
    'Иной вид ДТП': 'Другое'
}

df['DTPV'] = df['DTPV'].replace(replace_map)
sorted(df['DTPV'].unique())

['Другое',
 'Наезд на велосипедиста',
 'Наезд на животное',
 'Наезд на пешехода',
 'Наезд на препятствие',
 'Наезд на человека',
 'Прочее',
 'Столкновение']

In [284]:
le = LabelEncoder()
df['DTPV_cat'] = le.fit_transform(df['DTPV'])

--- 

__6. k_ul — street significance__

In [285]:
sorted(df['k_ul'].unique())

['Велосипедные дорожки',
 'Вне НП',
 'Второстепенные улицы в жилой застройке (переулки)',
 'Главные улицы',
 'Иные места',
 'Магистральные дороги',
 'Магистральные улицы общегородского значения',
 'Магистральные улицы районного значения',
 'Назначение неизвестно',
 'Основные улицы',
 'Основные улицы в жилой застройке',
 'Парковые дороги',
 'Парковые дороги, пешеходные улицы (пешеходные зоны)',
 'Пешеходные улицы',
 'Поселковые дороги',
 'Проезды',
 'Улицы и дороги местного значения в жилой застройке',
 'Улицы и дороги местного значения в производственных, промышленных и коммунально-складских зонах',
 'Улицы и дороги местного значения научно-производственных промышленных и коммунально-складских районов',
 'Хозяйственные проезды, скотопрогоны']

In [286]:
replace_map = {
    # Highways and main streets
    'Магистральные дороги': 'Магистральная дорога',
    
    'Магистральные улицы общегородского значения': 'Магистральная улица',
    'Магистральные улицы районного значения': 'Магистральная улица',
    'Главные улицы': 'Магистральная улица',
    'Основные улицы': 'Магистральная улица',
    'Основные улицы в жилой застройке': 'Магистральная улица',

    # Local streets
    'Второстепенные улицы в жилой застройке (переулки)': 'Улица местного значения',
    'Улицы и дороги местного значения в жилой застройке': 'Улица местного значения',
    'Улицы и дороги местного значения в производственных, промышленных и коммунально-складских зонах': 'Улица местного значения',
    'Улицы и дороги местного значения научно-производственных промышленных и коммунально-складских районов': 'Улица местного значения',
    'Проезды': 'Улица местного значения',
    'Поселковые дороги': 'Улица местного значения',
    'Хозяйственные проезды, скотопрогоны': 'Улица местного значения',

    # Pedestrian areas and parks
    'Пешеходные улицы': 'Пешеходная зона',
    'Парковые дороги': 'Пешеходная зона',
    'Парковые дороги, пешеходные улицы (пешеходные зоны)': 'Пешеходная зона',
    'Велосипедные дорожки': 'Пешеходная зона',

    # Other
    'Вне НП': 'Вне НП',
    'Назначение неизвестно': 'Не указано',
    'Иные места': 'Другое'
}

df['k_ul'] = df['k_ul'].replace(replace_map)
sorted(df['k_ul'].unique())

['Вне НП',
 'Другое',
 'Магистральная дорога',
 'Магистральная улица',
 'Не указано',
 'Пешеходная зона',
 'Улица местного значения']

To create the final datasets in our task, we need to filter out observations that occurred outside the city limits, as accidents in populated areas happen under completely different conditions and cannot be compared to those on toll roads.

As we know the variable `NP`, or populated area, turned out to be completely useless because it showed the nearest populated area to the accident, rather than whether the accident occurred in a populated area or not. However, we still have the `k_ul` variable, which accurately reflects where the accident occurred. We'll extract this information into a separate binary feature `out_of_town` before transforming `k_ul` into a categorical variable, since its original meaning will no longer be accessible after encoding.

In [287]:
df['out_of_town'] = df['k_ul'].isin([
    'Вне НП',
    'Не указано',
    'Магистральная дорога'
]).astype(int)

In [288]:
le = LabelEncoder()
df['k_ul_cat'] = le.fit_transform(df['k_ul'])

---

__7. spog — weather__

In [289]:
sorted(df['spog'].unique())

['Дождь',
 'Метель',
 'Пасмурно',
 'Снегопад',
 'Температура выше +30С',
 'Температура ниже -30С',
 'Туман',
 'Ураганный ветер',
 'Ясно']

In [290]:
replace_map = {
    # Precipitation
    'Дождь': 'Осадки',
    'Снегопад': 'Осадки',
    'Метель': 'Осадки',

    # Poor visibility
    'Туман': 'Плохая видимость',
    'Пасмурно': 'Плохая видимость',

    # Extreme temperatures
    'Температура выше +30С': 'Жара',
    'Температура ниже -30С': 'Мороз',

    # Wind
    'Ураганный ветер': 'Сильный ветер',

    # Good weather
    'Ясно': 'Хорошая погода'
}

df['spog'] = df['spog'].replace(replace_map)
sorted(df['spog'].unique())

['Жара',
 'Мороз',
 'Осадки',
 'Плохая видимость',
 'Сильный ветер',
 'Хорошая погода']

In [291]:
le = LabelEncoder()
df['spog_cat'] = le.fit_transform(df['spog'])

---

__8. OBJ_DTP — road network objects near RTA__

In [292]:
sorted(df['OBJ_DTP'].unique())

['АЗС',
 'Автовокзал (автостанция)',
 'Автостоянка (отделенная от проезжей части)',
 'Административные здания',
 'Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань)',
 'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)',
 'Жилые дома индивидуальной застройки',
 'Зоны отдыха',
 'Иная образовательная организация',
 'Иное образовательное учреждение',
 'Иной объект',
 'Кладбище',
 'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта)',
 'Ледовая переправа',
 'Ледовая переправа (официально открытая и оборудованная);',
 'Лечебные учреждения',
 'Медицинские (лечебные) организации',
 'Многоквартирные жилые дома',
 'Мост',
 'Надземный пешеходный переход',
 'Нерегулируемое пересечение с круговым движением',
 'Нерегулируемый ж/д переезд',
 'Нерегулируемый перекрёсток',
 'Нерегулируемый пешеходный переход',
 'Объект (здание, сооружение) религиозного культа',
 'Объект строительст

In [293]:
replace_map = {
    # Education
    'Школа либо иная детская (в т.ч. дошкольная) организация': 'Образование',
    'Школа либо иное детское (в т.ч. дошкольное) учреждение': 'Образование',
    'Иная образовательная организация': 'Образование',
    'Иное образовательное учреждение': 'Образование',

    # Medicine
    'Лечебные учреждения': 'Медицина',
    'Медицинские (лечебные) организации': 'Медицина',

    # Residential development
    'Жилые дома индивидуальной застройки': 'Жилая застройка',
    'Многоквартирные жилые дома': 'Жилая застройка',

    # Trade and food
    'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта)': 'Торговля',
    'Одиночный торговый объект, являющийся местом притяжения транспорта и (или) пешеходов': 'Торговля',
    'Объект торговли, общественного питания на автодороге вне НП': 'Торговля',

    # Transport facilities
    'АЗС': 'Транспортная инфраструктура',
    'Автовокзал (автостанция)': 'Транспортная инфраструктура',
    'Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань)': 'Транспортная инфраструктура',
    'Остановка маршрутного такси': 'Остановка',
    'Остановка общественного транспорта': 'Остановка',
    'Остановка трамвая': 'Остановка',
    'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)': 'Транспортная инфраструктура',
    'Автостоянка (отделенная от проезжей части)': 'Транспортная инфраструктура',
    'Пункт весового контроля': 'Транспортная инфраструктура',

    # Road infrastructure
    'Регулируемый перекрёсток': 'Дорожная инфраструктура',
    'Нерегулируемый перекрёсток': 'Дорожная инфраструктура',
    'Нерегулируемое пересечение с круговым движением': 'Дорожная инфраструктура',
    'Регулируемый пешеходный переход': 'Дорожная инфраструктура',
    'Нерегулируемый пешеходный переход': 'Дорожная инфраструктура',
    'Подземный пешеходный переход': 'Дорожная инфраструктура',
    'Надземный пешеходный переход': 'Дорожная инфраструктура',
    'Регулируемый ж/д переезд с дежурным': 'Дорожная инфраструктура',
    'Регулируемый ж/д переезд без дежурного': 'Дорожная инфраструктура',
    'Нерегулируемый ж/д переезд': 'Дорожная инфраструктура',
    'Мост': 'Дорожная инфраструктура',
    'Эстакада, путепровод': 'Дорожная инфраструктура',
    'Тоннель': 'Дорожная инфраструктура',
    'СП ДПС (КПМ)': 'Дорожная инфраструктура',

    # Other
    'Объект (здание, сооружение) религиозного культа': 'Прочее',
    'Административные здания': 'Прочее',
    'Зоны отдыха': 'Прочее',
    'Кладбище': 'Прочее',
    'Объект строительства': 'Прочее',
    'Производственное предприятие': 'Прочее',
    'Территориальное подразделение МВД России (либо его структурное подразделение)': 'Прочее',
    'Спортивные и развлекательные объекты': 'Прочее',
    'Иной объект': 'Прочее',

    # Ice crossing
    'Ледовая переправа': 'Ледовая переправа',
    'Ледовая переправа (официально открытая и оборудованная);': 'Ледовая переправа',
    'Стихийно возникшая (не предусмотренная) ледовая переправа': 'Ледовая переправа',

    # Absence of objects
    'Отсутствие в непосредственной близости объектов УДС и объектов притяжения': 'Нет объектов'
}

df['OBJ_DTP'] = df['OBJ_DTP'].replace(replace_map)
sorted(df['OBJ_DTP'].unique())

['Дорожная инфраструктура',
 'Жилая застройка',
 'Ледовая переправа',
 'Медицина',
 'Нет объектов',
 'Образование',
 'Остановка',
 'Прочее',
 'Торговля',
 'Транспортная инфраструктура']

In [294]:
le = LabelEncoder()
df['OBJ_DTP_cat'] = le.fit_transform(df['OBJ_DTP'])

---
__9. factor — factors influencing the traffic__

In [295]:
sorted(df['factor'].unique())

['Изменение в режиме движения вследствие согласованного массового мероприятия',
 'Конструктивное сужение проезжей части вследствие уменьшения количества полос движения',
 'Наличие  пересечения (примыкания), не предусмотренного проектной документацией',
 'Наличие на проезжей части неисправного ТС, мешающего движению',
 'Наличие на проезжей части, обочине или тротуаре торгового или иного объекта, затрудняющего движение или парковку транспорта и (или) движение пешеходов',
 'Наличие на тротуарах (пешеходных дорожках, обочинах) припаркованного  транспорта,   затрудняющего движение пешеходов',
 'Наличие пересечения (примыкания), не предусмотренного проектной документацией» (не применяется с 12.2020)',
 'Нарушение режима движения вследствие несогласованного массового мероприятия',
 'Несоответствие габарита моста (путепровода) ширине проезжей части  на подходах к нему',
 "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНи

In [296]:
replace_map = {
    # Traffic disruptions due to events
    'Изменение в режиме движения вследствие согласованного массового мероприятия': 'Массовое мероприятие',
    'Нарушение режима движения вследствие несогласованного массового мероприятия': 'Массовое мероприятие',
    'Режим движения изменён вследствие проведения спецмероприятия': 'Массовое мероприятие',

    # Road accident as a cause
    'Режим движения нарушен вследствие ранее произошедшего ДТП': 'ДТП ранее',

    # Road works/narrowing
    'Проведение дорожных и прочих работ без сужения проезжей части': 'Работы без сужения',
    'Сужение проезжей части вследствие проведения работ': 'Сужение из-за работ',
    'Сужение проезжей части припаркованным транспортом': 'Сужение из-за транспорта',
    'Конструктивное сужение проезжей части вследствие уменьшения количества полос движения': 'Конструктивное сужение',

    # Work on sidewalks/pedestrians
    'Проведение работ на тротуарах (пешеходных дорожках, обочинах),  затрудняющих движение пешеходов': 'Работы на тротуарах',
    'Неудовлетворительное состояние тротуара или пешеходной дорожки (в  т.ч. недостатки зимнего содержания), затрудняющие или препятствующие движению пешеходов': 'Проблемы на тротуаре',

    # Defective/abandoned vehicle
    'Наличие на проезжей части неисправного ТС, мешающего движению': 'ТС мешает движению',
    'Наличие на тротуарах (пешеходных дорожках, обочинах) припаркованного  транспорта,   затрудняющего движение пешеходов': 'ТС мешает движению',
    
    # Foreign objects
    'Наличие на проезжей части, обочине или тротуаре торгового или иного объекта, затрудняющего движение или парковку транспорта и (или) движение пешеходов': 'Посторонние объекты',
    'Оказание влияния на движение транспортного потока или отдельного транспортного средства, наличия или внезапного возникновения на проезжей части посторонних предметов (упавшее дерево или мачта освещения, осыпь камней, селевой вынос, груз с другого транспортного сред-ства или конструктивный элемент другого транспортного средства и т.д.)': 'Посторонние объекты',
    'Оказание влияния на движение транспортного потока или отдельного транспортного средства, наличия или внезапного появления на проезжей части животных': 'Животные/препятствия',

    # Intersections, junctions
    'Наличие  пересечения (примыкания), не предусмотренного проектной документацией': 'Нелегальное примыкание',
    'Наличие пересечения (примыкания), не предусмотренного проектной документацией» (не применяется с 12.2020)': 'Нелегальное примыкание',

    # Road non-compliance with standards
    "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНиП 2.05.02.-85 'Автомобильные дороги')": 'Несоответствие категории',
    "Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СНиП 2.05.02.-85 ?Автомобильные дороги?)": 'Несоответствие категории',
    'Несоответствие категории дороги (фактических геометрических характеристик) интенсивности движения (исходя из требований СП 34.13330.2012 «Свод правил. Автомобильные дороги. Актуализированная редакция СНиП 2.05.02-85*»)': 'Несоответствие категории',
    'Несоответствие параметров дороги, в т.ч. геометрических, её категории на месте ДТП или на подходах к нему (величина радиуса поворота, ширина полос движения, обочин, крутизна откосов насыпи, отсутствие разделительной полосы, отсутствие виража и т.д.)': 'Несоответствие категории',
    'Несоответствие габарита моста (путепровода) ширине проезжей части  на подходах к нему': 'Несоответствие габаритов',

    # Electricity supply
    'Отключение электроснабжения на данном элементе УДС (улице, дороге, площади, дворе и т.д.) в целом': 'Отключение электроснабжения',
    'Отключение электроснабжения на данном элементе УДС (улице, дороге, площади, дворе и т.д.) в целом (не применяется с 12.2020)': 'Отключение электроснабжения',
    'Отключение электроснабжения целиком или частично в муниципальном образовании (районе, городе, районе города, посёлке, селе, деревне, ауле и т.д.)': 'Отключение электроснабжения',
    'Отключение электроснабжения целиком или частично в муниципальном образовании  (районе, городе, районе  города, посёлке, селе, деревне, ауле и т.д.)': 'Отключение электроснабжения',

    # Traffic lanes
    'Отсутствие дополнительных полос для движения (в т.ч. переходно-скоростных полос) в необходимых местах': 'Отсутствие полос',
    'Отсутствие дополнительных полос для движения (в т.ч. переходно-скоростных полос) в необходимых местах (не применяется с 12.2020)': 'Отсутствие полос',

    # Traffic lights
    "Работа светофора на регулируемом перекрёстке или регулируемом пешеходном переходе в режиме 'жёлтого мигания'": 'Режим светофора',

    # Speed ​​cameras
    'Участок, контролируемый камерами автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',
    'Участок, контролируемый стационарными камерами автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',
    'Участок, контролируемый стационарными камерами автоматической фотовидеофиксации нарушений ПДД, не обозначенный соответствующим предупреждающим знаком': 'Камера (не обозначена)',
    'Участок, контролируемый мобильными автоматической фотовидеофиксации нарушений ПДД': 'Камера (мобильная)',
    'Участок, контролируемый передвижными средствами автоматической фотовидеофиксации нарушений ПДД': 'Камера (мобильная)',
    'Участок, контролируемый камерой автоматической фотовидеофиксации нарушений ПДД, не обозначенный соответствующим предупреждающим знаком': 'Камера (не обозначена)',

    'Участок, контролируемый камерой автоматической фотовидеофиксации нарушений ПДД, обозначенный соответствующим предупреждающим знаком': 'Камера (обозначена)',

    # Speed ​​bumps
    'Участок, оборудованный искусственными неровностями': 'Искусственные неровности',
    'Участок, оборудованный искусственными неровностями и обозначенный соответствующими дорожными знаками и разметкой': 'Искусственные неровности',
    'Участок, оборудованный искусственными неровностями, не обозначенный соответствующими дорожными знаками и (или) разметкой': 'Искусственные неровности',
    'Участок, оборудованный шумовыми полосами': 'Шумовые полосы',

    # Other
    'Сведения отсутствуют': 'Сведения отсутствуют'
}

df['factor'] = df['factor'].replace(replace_map)
sorted(df['factor'].unique())

['ДТП ранее',
 'Животные/препятствия',
 'Искусственные неровности',
 'Камера (мобильная)',
 'Камера (не обозначена)',
 'Камера (обозначена)',
 'Конструктивное сужение',
 'Массовое мероприятие',
 'Нелегальное примыкание',
 'Несоответствие габаритов',
 'Несоответствие категории',
 'Отключение электроснабжения',
 'Отсутствие полос',
 'Посторонние объекты',
 'Проблемы на тротуаре',
 'Работы без сужения',
 'Работы на тротуарах',
 'Режим светофора',
 'Сведения отсутствуют',
 'Сужение из-за работ',
 'Сужение из-за транспорта',
 'ТС мешает движению',
 'Шумовые полосы']

In [297]:
le = LabelEncoder()
df['factor_cat'] = le.fit_transform(df['factor'])

---

__10. fatal traffic violation(s) — ...__

It's a multiclass categorial feature.

In [298]:
df['fatal traffic violation(s)'] = df['fatal traffic violation(s)'].apply(ast.literal_eval)       # from string "{}" to real set {}

all_violations = set()

for violations in df['fatal traffic violation(s)']:
    all_violations.update(violations)

unique_violations = sorted(all_violations)
for i, v in enumerate(unique_violations, 1):
    print(f"{i}. {v}")

1. Выезд на полосу встречного движения
2. Выезд на полосу встречного движения в местах, где это запрещено
3. Выезд на полосу встречного движения с разворотом, поворотом налево или объездом препятствия
4. Выезд на трамвайные пути встречного направления
5. Движение вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины
6. Движение вдоль проезжей части при наличии и удовлетворительном состоянии тротуара
7. Движение во встречном направлении по дороге с односторонним движением
8. Другие нарушения ПДД водителем
9. Другие нарушения ПДД водителями (не применяется с 12.2020)
10. Игра на проезжей части
11. Иные нарушения
12. Нарушение правил буксировки
13. Нарушение правил обгона
14. Нарушение правил остановки и стоянки
15. Нарушение правил перевозки крупногабаритных и тяжеловесных грузов
16. Нарушение правил перевозки людей
17. Нарушение правил перевозки опасных грузов
18. Нарушение правил перестроения
19. Нарушение правил погрузки, выгрузки и

In [ ]:
replace_map = {
    # Oncoming lane, exit
    'Выезд на полосу встречного движения': 'Выезд на встречку',
    'Выезд на полосу встречного движения в местах, где это запрещено': 'Выезд на встречку',
    'Выезд на полосу встречного движения с разворотом, поворотом налево или объездом препятствия': 'Выезд на встречку',
    'Выезд на трамвайные пути встречного направления': 'Выезд на встречку',
    'Движение во встречном направлении по дороге с односторонним движением': 'Выезд на встречку',

    # Crossings, pedestrians
    'Непредоставление преимущества в движении пешеходу': 'Пешеходы',
    'Переход через проезжую часть в неустановленном месте (при наличии в зоне видимости перекрёстка)': 'Пешеходы',
    'Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода': 'Пешеходы',
    'Переход проезжей части в запрещённом месте (оборудованном пешеходными  ограждениями)': 'Пешеходы',
    'Нахождение на проезжей части без цели её перехода': 'Пешеходы',
    'Ожидание маршрутного ТС на проезжей части, вне посадочных площадок, тротуаров и обочин (при их наличии)': 'Пешеходы',
    'Игра на проезжей части': 'Пешеходы',
    'Ходьба вдоль проезжей части при наличии и удовлетворительном состоянии тротуара': 'Пешеходы',
    'Ходьба вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины': 'Пешеходы',
    'Неожиданный выход из-за ТС': 'Пешеходы',
    'Неожиданный выход из-за сооружений (деревьев)': 'Пешеходы',
    'Неожиданный выход из-за стоящего ТС': 'Пешеходы',

    # Speed ​​and distance
    'Превышение установленной скорости движения': 'Скорость',
    'Неправильный выбор дистанции': 'Скорость',
    'Несоответствие скорости конкретным условиям движения': 'Скорость',
    'Несоблюдение бокового интервала': 'Скорость',

    # Traffic lights, regulation
    'Неподчинение сигналам регулирования': 'Регулирование',
    'Нарушение требований сигналов светофора': 'Регулирование',
    'Нарушение требований сигналов регулировщика': 'Регулирование',

    # Markings and signs
    'Нарушение требований дорожных знаков': 'Разметка/знаки',
    'Нарушение требований линий разметки': 'Разметка/знаки',

    # Overtaking and changing lanes
    'Нарушение правил обгона': 'Манёвры',
    'Нарушение правил перестроения': 'Манёвры',
    'Разворот в местах, где разворот запрещён': 'Манёвры',
    'Невыполнение требований обеспечения безопасности при начале движения': 'Манёвры',

    # Drive
    'Несоблюдение очередности проезда': 'Проезд',
    'Несоблюдение очередности проезда перекрестков': 'Проезд',
    'Нарушение правил при круговом движении': 'Проезд',
    'Нарушение правил проезда ж/д переездов': 'Проезд',
    'Нарушение правил проезда пешеходного перехода': 'Проезд',
    'Нарушение правил проезда остановок трамвая': 'Проезд',
    'Движение вдоль проезжей части попутного направления вне населенного пункта при удовлетворительном состоянии обочины': 'Проезд',
    'Движение вдоль проезжей части при наличии и удовлетворительном состоянии тротуара': 'Проезд',

    # Signals
    'Неподача или неправильная подача сигналов': 'Сигналы',
    'Неподача или неправильная подача сигналов поворота': 'Сигналы',

    # Stopping/Parking
    'Нарушение правил остановки и стоянки': 'Остановка/стоянка',
    'Стоянка на проезжей части или обочине без освещения': 'Остановка/стоянка',

    # Technical problems
    'Эксплуатация ТС с техническими неисправностями, при которых запрещается их эксплуатация': 'Технеисправности',
    'Эксплуатация технически неисправного ТС (проставляется в случае ДТП по причине технеисправности)': 'Технеисправности',

    # Shipping
    'Нарушение правил перевозки людей': 'Перевозка',
    'Нарушение правил перевозки опасных грузов': 'Перевозка',
    'Нарушение правил перевозки крупногабаритных и тяжеловесных грузов': 'Перевозка',
    'Нарушение правил погрузки, выгрузки и крепления грузов': 'Перевозка',
    'Нарушение правил движения тяжеловесного и(или) крупногабаритного ТС(превышение допустимых габаритов ТС, превышением допустимой массы ТС или допустимой нагрузки на ось ТС)': 'Перевозка',

    # Bike
    'Управление велосипедом с неисправной тормозной системой': 'Велосипед',
    'Управление велосипедом, не оснащённым светоотражающими элементами': 'Велосипед',
    'Пересечение велосипедистом проезжей части по пешеходному переходу': 'Велосипед',

    # Public transport and minibuses
    'Нарушение правил пользования общественным транспортом': 'Общественный транспорт',

    # Other
    'Сведения отсутствуют': 'Не указано',
    'Другие нарушения ПДД водителем': 'Другое',
    'Другие нарушения ПДД водителями (не применяется с 12.2020)': 'Другое',
    'Иные нарушения': 'Другое',
    'Создание помех для водителя в управлении транспортным средством': 'Другое',
    'Ослепление светом фар': 'Другое',
    'Оставление движущегося транспортного средства (выход или выпрыгивание на ходу и т.д.)': 'Другое',
    'Передвижение по проезжей части на скейтборде (роликовых коньках и т.д.)': 'Другое',
    'Переход проезжей части перед приближающимся ТС с включёнными проблесковым маячком синего (сине-красного) цвета и звуковым сигналом': 'Другое',
    'Неповиновение или сопротивление находящемуся при исполнении служебных обязанностей сотруднику правоохранительных органов или военнослужащему при остановке или задержании ТС': 'Другое',
    'Непредоставление преимущества в движении ТС, имеющему нанесенные на наружные поверхности специальные цветографические схемы, надписи и обозначения, с одновременно включённым проблесковым маячком синего цвета и специальным звуковым сигналом': 'Другое',
    'Нарушение правил буксировки': 'Другое',
    'Несоблюдение условий, разрешающих движение транспорта задним ходом': 'Другое',
    'Нарушение правил пользования внешними световыми приборами': 'Другое',
    'Нарушение правил расположения ТС на проезжей части': 'Другое'
}


def map_violation_set(violation_set):
    return set(replace_map.get(v, v) for v in violation_set)

df['fatal traffic violation(s)'] = df['fatal traffic violation(s)'].apply(map_violation_set)

In [300]:
all_violations = set()

for violations in df['fatal traffic violation(s)']:
    all_violations.update(violations)

unique_violations = sorted(all_violations)
for i, v in enumerate(unique_violations, 1):
    print(f"{i}. {v}")

1. Велосипед
2. Выезд на встречку
3. Другое
4. Манёвры
5. Общественный транспорт
6. Остановка/стоянка
7. Перевозка
8. Пешеходы
9. Проезд
10. Разметка/знаки
11. Регулирование
12. Сигналы
13. Скорость
14. Технеисправности


In [301]:
print(f"Previous num of features: {df.shape[1]}\n")

mlb = MultiLabelBinarizer()
violations_encoded = pd.DataFrame(
    mlb.fit_transform(df['fatal traffic violation(s)']),
    columns=mlb.classes_,
    index=df.index
)

df = pd.concat([df, violations_encoded], axis=1)

print(f"Current num of features: {df.shape[1]}")

Previous num of features: 77

Current num of features: 91


---

__11. osv — lighting__

In [302]:
sorted(df['osv'].unique())

['В темное время суток, освещение включено',
 'В темное время суток, освещение не включено',
 'В темное время суток, освещение отсутствует',
 'Не установлено',
 'Светлое время суток',
 'Сумерки']

In [303]:
replace_map = {
    'Светлое время суток': 'День',
    'Сумерки': 'Сумерки',
    'В темное время суток, освещение включено': 'Ночь с освещением',
    'В темное время суток, освещение не включено': 'Ночь без освещения',
    'В темное время суток, освещение отсутствует': 'Ночь без освещения',
    'Не установлено': 'Не указано'
}

df['osv'] = df['osv'].replace(replace_map)
sorted(df['osv'].unique())

['День', 'Не указано', 'Ночь без освещения', 'Ночь с освещением', 'Сумерки']

In [304]:
le = LabelEncoder()
df['osv_cat'] = le.fit_transform(df['osv'])

---

__12. sdor — road network objects at the scene of RTA__

In [305]:
sorted(df['sdor'].unique())

['АЗС',
 'Автостоянка (не отделённая от проезжей части)',
 'Автостоянка (отделенная от проезжей части)',
 'Внутридворовая территория',
 'Выезд с прилегающей территории',
 'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)',
 'Иное место',
 'Ледовая переправа',
 'Ледовая переправа(официально открытая и оборудованная)',
 'Место для перегона скота',
 'Мост, эстакада, путепровод',
 'Нерегулируемое пересечение с круговым движением',
 'Нерегулируемый ж/д переезд',
 'Нерегулируемый перекрёсток неравнозначных улиц (дорог)',
 'Нерегулируемый перекрёсток равнозначных улиц (дорог)',
 'Нерегулируемый пешеходный переход',
 'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения',
 'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации',
 'Остановка маршрутного такси',
 

In [306]:
replace_map = {
    # Pedestrian crossings
    'Нерегулируемый пешеходный переход': 'Пешеходный переход (нерегулируемый)',
    'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения': 'Пешеходный переход (нерегулируемый)',
    'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации': 'Пешеходный переход (нерегулируемый)',

    'Регулируемый пешеходный переход': 'Пешеходный переход (регулируемый)',
    'Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения': 'Пешеходный переход (регулируемый)',
    'Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации': 'Пешеходный переход (регулируемый)',

    # Railway crossings
    'Регулируемый ж/д переезд с дежурным': 'Ж/д переезд (регулируемый)',
    'Регулируемый ж/д переезд без дежурного': 'Ж/д переезд (регулируемый)',

    'Нерегулируемый ж/д переезд': 'Ж/д переезд (нерегулируемый)',

    # Crossroads
    'Регулируемый перекресток': 'Перекрёсток (регулируемый)',

    'Нерегулируемый перекрёсток неравнозначных улиц (дорог)': 'Перекрёсток (нерегулируемый)',
    'Нерегулируемый перекрёсток равнозначных улиц (дорог)': 'Перекрёсток (нерегулируемый)',

    'Нерегулируемое пересечение с круговым движением': 'Перекрёсток (круговое)',

    # Public transport
    'Остановка маршрутного такси': 'Остановка',
    'Остановка общественного транспорта': 'Остановка',
    'Остановка трамвая': 'Остановка',

    # Ice crossing
    'Ледовая переправа': 'Ледовая переправа',
    'Ледовая переправа(официально открытая и оборудованная)': 'Ледовая переправа',
    'Стихийно возникшая (не предусмотренная) ледовая переправа': 'Ледовая переправа',

    # Bridges and tunnels
    'Мост, эстакада, путепровод': 'Мост/эстакада/тоннель',
    'Подход к мосту, эстакаде, путепроводу': 'Мост/эстакада/тоннель',
    'Тоннель': 'Мост/эстакада/тоннель',

    # Parking, parking lot, adjacent territory
    'Автостоянка (отделенная от проезжей части)': 'Стоянка',
    'Автостоянка (не отделённая от проезжей части)': 'Стоянка',
    'АЗС': 'Стоянка',
    'Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)': 'Стоянка',
    
    'Выезд с прилегающей территории': 'Прилегающая территория',
    'Внутридворовая территория': 'Прилегающая территория',

    # Pedestrian areas
    'Пешеходная зона': 'Пешеходная зона',
    'Тротуар, пешеходная дорожка': 'Пешеходная зона',

    # Other
    'Место для перегона скота': 'Прочее',
    'Перегон (нет объектов на месте ДТП)': 'Прочее',
    'Иное место': 'Прочее'
}


df['sdor'] = df['sdor'].replace(replace_map)
sorted(df['sdor'].unique())

['Ж/д переезд (нерегулируемый)',
 'Ж/д переезд (регулируемый)',
 'Ледовая переправа',
 'Мост/эстакада/тоннель',
 'Остановка',
 'Перекрёсток (круговое)',
 'Перекрёсток (нерегулируемый)',
 'Перекрёсток (регулируемый)',
 'Пешеходная зона',
 'Пешеходный переход (нерегулируемый)',
 'Пешеходный переход (регулируемый)',
 'Прилегающая территория',
 'Прочее',
 'Стоянка']

In [307]:
le = LabelEncoder()
df['sdor_cat'] = le.fit_transform(df['sdor'])

---

__13. bodily harm — ...__

Also has multiple classes.

In [308]:
df['bodily harm'] = df['bodily harm'].apply(ast.literal_eval)       # from string "{}" to real set {}

all_harms = set()

for harm in df['bodily harm']:
    all_harms.update(harm)

unique_harms = sorted(all_harms)
for i, v in enumerate(unique_harms, 1):
    print(f"{i}. {v}")

1. Не пострадал
2. Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)
3. Получил телесные повреждения с показанием к лечению в медицинских организациях, фактически лечение не проходил, к категории раненый не относится
4. Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится
5. Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)
6. Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара
7. Раненый, находящийся (находившийся) на стационарном лечении
8. Скончался в течение 1 суток
9. Скончался в течение 10 суток
10. Скончался в течение 11 суток
11. Скончался в течение 12 суток
12. Скончался в течение 13 суток
13. Скончался в течение 14 суток
14. Скончался в течение 15 суток
15. Скончался в течение 16 суто

In [309]:
replace_map = {
    # No harm
    'Не пострадал': 'Не пострадал',

    # Minor injuries (not wounds)
    'Получил телесные повреждения с показанием к лечению в медицинских организациях, фактически лечение не проходил, к категории раненый не относится': 'Лёгкая травма',
    'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится': 'Лёгкая травма',

    # Moderate injuries (not hospitalization)
    'Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)': 'Травма (без госпитализации)',
    'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)': 'Травма (без госпитализации)',
    'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара': 'Травма (без госпитализации)',

    # Severe injuries (hospitalization)
    'Раненый, находящийся (находившийся) на стационарном лечении': 'Травма (госпитализация)',

    # Death on the spot
    'Скончался на месте ДТП до приезда скорой медицинской помощи': 'Смерть на месте',
    'Скончался на месте ДТП по прибытию скорой медицинской помощи, но до транспортировки в мед. организацию': 'Смерть на месте',
    'Скончался на месте ДТП по прибытию скорой медицинской помощи, но до транспортировки в медицинское учреждение': 'Смерть на месте',

    # Death during transportation
    'Скончался при транспортировке': 'Смерть при транспортировке',

    # Death
    'Скончался в течение 1 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 2 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 3 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 4 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 5 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 6 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 7 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 8 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 9 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 10 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 11 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 12 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 13 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 14 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 15 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 16 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 17 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 18 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 19 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 20 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 21 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 22 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 23 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 24 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 25 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 26 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 27 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 28 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 29 суток': 'Смерть в течение 30 суток',
    'Скончался в течение 30 суток': 'Смерть в течение 30 суток',
}

def map_violation_set(violation_set):
    return set(replace_map.get(v, v) for v in violation_set)

df['bodily harm'] = df['bodily harm'].apply(map_violation_set)

In [310]:
all_harms = set()

for harm in df['bodily harm']:
    all_harms.update(harm)

unique_harms = sorted(all_harms)
for i, v in enumerate(unique_harms, 1):
    print(f"{i}. {v}")

1. Лёгкая травма
2. Не пострадал
3. Смерть в течение 30 суток
4. Смерть на месте
5. Смерть при транспортировке
6. Травма (без госпитализации)
7. Травма (госпитализация)


In [311]:
print(f"Previous num of features: {df.shape[1]}\n")

mlb = MultiLabelBinarizer()
harms_encoded = pd.DataFrame(
    mlb.fit_transform(df['bodily harm']),
    columns=mlb.classes_,
    index=df.index
)

df = pd.concat([df, harms_encoded], axis=1)

print(f"Current num of features: {df.shape[1]}")

Previous num of features: 93

Current num of features: 100


---

In [312]:
# drop raw categorial features
drop_columns.extend(cat_cols)

## Drop unnecessary columns

In [313]:
df = df.drop(drop_columns, axis=1)

print(f"Features left: {df.shape[1]}, in the begining was: {features_number}")

Features left: 79, in the begining was: 91


In [314]:
df.columns

Index(['date', 'COORD_L', 'COORD_W', 'dor', 'dor_k', 's_dtp', 'KTS', 'KUCH',
       'kartId', 'POG', 'RAN', 'time', 'any TV technical failures',
       'any non-private vehicles', 'any russian vehicles',
       'any white vehicles', 'any black vehicles', 'any colored vehicles',
       'any drunk drivers', 'any female drivers', 'any escaped participants',
       'any concomitant traffic violations',
       'any injuries without safety belt', 'num drunk participants',
       'num child restraints', 'num cyclists', 'num pedestrians',
       'min vehicles age', 'max vehicles age', 'avg vehicles age',
       'num A-class vehicles', 'num B-class vehicles', 'num C-class vehicles',
       'num D-class vehicles', 'num E-class vehicles', 'num S-class vehicles',
       'num non-car vehicles', 'num front-wheel-drive vehicles',
       'num rear-wheel-drive vehicles', 'num four-wheel-drive vehicles',
       'num guilty objects', 'share guilty vehicles',
       'num fatal traffic violations', 'guilty

## Processing the data gaps (part 2) with ML algorithms

After initial processing of gaps there are still columns with NaN. In this part we will fill them using machine learning methods.

In [315]:
cols_with_gaps = df.columns[df.isna().any()].tolist()

print('Columns for proceed:\n')
for col in cols_with_gaps:
    gaps = df[col].isna().sum()
    print(f"'{col}' — {gaps} gaps")

print(f"\nCurrent num of obs: {df.shape[0]}")

Columns for proceed:

'dor_k' — 11652 gaps
'min vehicles age' — 6208 gaps
'max vehicles age' — 6208 gaps
'avg vehicles age' — 6208 gaps

Current num of obs: 473194


In [316]:
df_ml = df.copy()

# encoding 'dor'
le = LabelEncoder()
df_ml['dor_encoded'] = le.fit_transform(df_ml['dor'])

In [317]:
# train models to dill the gaps on these data
df_train = df_ml.dropna()
print(f"Num of obs in train DataFrame: {df_train.shape[0]}")

Num of obs in train DataFrame: 455568


#### **'dor_k'**

In [318]:
df['dor_k'].unique()

array([ 5.,  6.,  7.,  3.,  4.,  2.,  1.,  8., nan])

Analysis shows that `'dor_k'` is a categorical preproceed feature.

In [319]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

df_knn = df_train[['dor_encoded', 'dor_z_cat', 'COORD_L', 'COORD_W', 'dor_k']]

X = df_knn[['dor_encoded', 'dor_z_cat', 'COORD_L', 'COORD_W']]
y = df_knn['dor_k'].astype(int)

# normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_val)

print("Classification report:\n", classification_report(y_val, y_pred))

Classification report:
               precision    recall  f1-score   support

           1       0.62      0.67      0.65      7249
           2       0.55      0.47      0.51      5393
           3       0.48      0.34      0.40      3721
           4       0.72      0.77      0.74     22801
           5       0.79      0.82      0.81     27332
           6       0.80      0.80      0.80     19786
           7       0.66      0.50      0.57      2172
           8       0.74      0.56      0.64      2660

    accuracy                           0.74     91114
   macro avg       0.67      0.62      0.64     91114
weighted avg       0.73      0.74      0.73     91114



In [320]:
df_missing = df_ml[df_ml['dor_k'].isna()]

X_missing = df_missing[['dor_encoded', 'dor_z_cat', 'COORD_L', 'COORD_W']]
X_missing_scaled = scaler.transform(X_missing)

predicted_dor_k = knn.predict(X_missing_scaled)

# Записываем результат обратно в основной DataFrame
df.loc[df_missing.index, 'dor_k'] = predicted_dor_k

In [321]:
gaps = df['dor_k'].isna().sum()
print(f"Column 'dor_k' has {gaps} gaps.")

Column 'dor_k' has 0 gaps.


#### **'min vehicles age', 'max vehicles age', 'avg vehicles age'**

In [322]:
df[['min vehicles age', 'max vehicles age', 'avg vehicles age']].dtypes

min vehicles age    float64
max vehicles age    float64
avg vehicles age    float64
dtype: object

In [323]:
features = [
    'KTS', 'KUCH', 'any non-private vehicles', 'any russian vehicles',
    'any drunk drivers', 'any escaped participants',
    'num A-class vehicles', 'num B-class vehicles', 'num C-class vehicles',
    'num D-class vehicles', 'num E-class vehicles', 'num S-class vehicles',
    'num non-car vehicles', 'num front-wheel-drive vehicles',
    'num rear-wheel-drive vehicles', 'num four-wheel-drive vehicles',
    'drivers avg experience', 'guilty drivers avg experience',
    'any TV technical failures',
    'spog_cat', 'osv_cat', 'factor_cat', 'out_of_town'
]

X = df_train[features]
y = df_train[['min vehicles age', 'max vehicles age', 'avg vehicles age']].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mae_min = mean_absolute_error(y_test.iloc[:, 0], y_pred[:, 0])
mae_max = mean_absolute_error(y_test.iloc[:, 1], y_pred[:, 1])
mae_avg = mean_absolute_error(y_test.iloc[:, 2], y_pred[:, 2])

print(f"MAE (min vehicles age): {mae_min:.2f}")
print(f"MAE (max vehicles age): {mae_max:.2f}")
print(f"MAE (avg vehicles age): {mae_avg:.2f}")
print(f"\nOverall MAE (all targets averaged): {mae:.2f}")

MAE (min vehicles age): 6.42
MAE (max vehicles age): 4.94
MAE (avg vehicles age): 5.29

Overall MAE (all targets averaged): 5.55


In [324]:
features = [
    'KTS', 'KUCH', 'any non-private vehicles', 'any russian vehicles',
    'any drunk drivers', 'any escaped participants',
    'num A-class vehicles', 'num B-class vehicles', 'num C-class vehicles',
    'num D-class vehicles', 'num E-class vehicles', 'num S-class vehicles',
    'num non-car vehicles', 'num front-wheel-drive vehicles',
    'num rear-wheel-drive vehicles', 'num four-wheel-drive vehicles',
    'drivers avg experience', 'guilty drivers avg experience',
    'any TV technical failures',
    'spog_cat', 'osv_cat', 'factor_cat', 'out_of_town'
]

df_missing = df_ml[df_ml[['min vehicles age', 'max vehicles age', 'avg vehicles age']].isna().any(axis=1)]
X_missing = df_missing[features]

y_pred = model.predict(X_missing)

y_pred_fixed = y_pred.copy()

for i in range(y_pred.shape[0]):
    min_val, max_val, avg_val = y_pred[i]

    min_val = min(min_val, avg_val, max_val)
    max_val = max(min_val, avg_val, max_val)
    avg_val = max(min_val, min(avg_val, max_val))

    y_pred_fixed[i] = [min_val, max_val, avg_val]


df.loc[df_missing.index, 'min vehicles age'] = y_pred_fixed[:, 0]
df.loc[df_missing.index, 'max vehicles age'] = y_pred_fixed[:, 1]
df.loc[df_missing.index, 'avg vehicles age'] = y_pred_fixed[:, 2]

In [325]:
gaps = df['min vehicles age'].isna().sum()
print(f"Column 'min vehicles age' has {gaps} gaps.")

gaps = df['max vehicles age'].isna().sum()
print(f"Column 'max vehicles age' has {gaps} gaps.")

gaps = df['avg vehicles age'].isna().sum()
print(f"Column 'avg vehicles age' has {gaps} gaps.")

Column 'min vehicles age' has 0 gaps.
Column 'max vehicles age' has 0 gaps.
Column 'avg vehicles age' has 0 gaps.


In [326]:
df.isna().any().sum()

0

## Rename the columns

In [327]:
new_column_names = {
    'DTPV': 'RTA type',
    'kartId': 'RTA number',
    'CHOM': 'traffic changes info',
    'dor_z': 'road significance',
    'factor': 'factors influenced traffic',
    'k_ul': 'street significance',
    'km': 'RTA road km',
    'm': 'RTA road m',
    'ndu': 'road network maintenance defects',
    'OBJ_DTP': 'RTA-adjacent road objects',
    'osv': 'lighting',
    's_pch': 'road surface condition',
    'sdor': 'RTA site road objects',
    'spog': 'weather',
    'KTS': 'RTA vehicle number',
    'KUCH': 'RTA participants number',
    'POG': 'number of deaths',
    'RAN': 'number of wounded',
    'time': 'RTA time',

    'Велосипед': 'bicycle_related',
    'Выезд на встречку': 'wrong_way_driving',
    'Другое': 'other_violation',
    'Манёвры': 'maneuvers',
    'Общественный транспорт': 'public_transport',
    'Остановка/стоянка': 'stopping_or_parking',
    'Перевозка': 'transport_violation',
    'Пешеходы': 'pedestrian_related',
    'Проезд': 'intersection_crossing',
    'Разметка/знаки': 'markings_or_signs',
    'Регулирование': 'traffic_control_violation',
    'Сигналы': 'signal_violation',
    'Скорость': 'speeding',
    'Технеисправности': 'technical_malfunction',
    'osv_cat': 'lighting_condition',
    'sdor_cat': 'road_object_type',
    'Лёгкая травма': 'minor_injury',
    'Не пострадал': 'no_injury',
    'Смерть в течение 30 суток': 'death_within_30_days',
    'Смерть на месте': 'death_at_scene',
    'Смерть при транспортировке': 'death_during_transport',
    'Травма (без госпитализации)': 'injury_no_hospitalization',
    'Травма (госпитализация)': 'injury_with_hospitalization'
}


df = df.rename(columns=new_column_names)

In [328]:
df.shape

(473194, 79)

In [329]:
df.to_csv('DTP_DATA_2025_PROCESSED.csv')